In [ ]:
import numpy as np
from kernels import (
    periodic_kernel,
    gaussian_kernel,
    exponential_kernel,
    linear_kernel
)

SAMPLE_POINTS = 300
SAMPLES = 2
X_RANGE = 3


def main():
    # Sample equidistant B
    x_b = np.linspace(-X_RANGE, X_RANGE, SAMPLE_POINTS)

    outputs = []

    for kernel, kernel_name in [
        (periodic_kernel, "periodic"),
        (gaussian_kernel, "gaussian"),
        (exponential_kernel, "exponential"),
        (linear_kernel, "linear"),
    ]:
        # Compute mean and covariance matrix
        mu = np.zeros(x_b.shape[0])
        k = kernel(x_b, x_b)

        # Sample from the multivariate Gaussian
        f_b = np.random.multivariate_normal(mu, k, (SAMPLES))

        for i in range(SAMPLES):
            outputs.append((f"{kernel_name} {i}", f_b[i]))

        k_at_0 = k[SAMPLES // 2 + 0 * SAMPLES // (2 * X_RANGE) - 1]
        k_at_15 = k[SAMPLES // 2 + int(1.5 * SAMPLES // (2 * X_RANGE)) - 1]
        k_at_neg1 = k[SAMPLES // 2 + -1 * SAMPLES // (2 * X_RANGE) - 1]
        outputs.append((f"k at 0 {kernel_name}", k_at_0))
        outputs.append((f"k at 1.5 {kernel_name}", k_at_15))
        outputs.append((f"k at -1 {kernel_name}", k_at_neg1))

    lines = []
    line = "x,mean,std,"
    for key, _ in outputs:
        line += f"{key},"

    line = line[:-1] + "\n"
    lines.append(line)

    for (x, index) in zip(x_b, range(SAMPLE_POINTS)):
        line = f"{x},0,1,"
        for _, values in outputs:
            line += f"{values[index]},"
        line = line[:-1] + "\n"
        lines.append(line)

    with open("gp/a_priori_samples.csv", "w") as file:
        file.writelines(lines)


if __name__ == "__main__":
    main()

In [ ]:
from scipy.spatial.distance import cdist
import numpy as np


def periodic_kernel(
    x, y,
    lengthscale: float = 1,
    period: float = 3,
):
    x = np.atleast_2d(x).T
    y = np.atleast_2d(y).T
    dists = cdist(x, y, metric="euclidean")

    return np.exp(
        -2 * (np.sin(np.pi / period * dists) / lengthscale) ** 2
    )


def linear_kernel(
    x, y,
    sigma_0: float = 20,
):
    x = np.atleast_2d(x).T
    y = np.atleast_2d(y).T
    return np.inner(x, y) + sigma_0**2


def gaussian_kernel(
    x, y,
    lengthscale: float = 1,
):
    x = np.atleast_2d(x).T
    y = np.atleast_2d(y).T
    dists = cdist(x, y, metric="sqeuclidean")

    return np.exp(-dists / (2 * lengthscale**2))


def exponential_kernel(
    x, y,
    lengthscale: float = 1,
):
    x = np.atleast_2d(x).T
    y = np.atleast_2d(y).T
    dists = cdist(x, y, metric="euclidean")

    return np.exp(-dists / lengthscale)